### Yelp Data Summary
The data collection stores one csv file for each zip code and it is placed in the "Output" folder.  The naming convention of these files is "YelpData_<SearchZipCode>.csv".  The code does the following:
    1 Gets a list of the files within the "Output" folder
    2 Loops through the files that start with "YelpData" and does the following:
      2.1 Read the file into Pandas DataFrame
      2.2 Summarize data; get total records
      2.3 Summarize data; get total records the different rating
      2.4 Summarize data; get total records for the different prices
    3 All of the summarized data are stored in DataFrame
    4 Summarized DataFrame is stored on disk; name "SummarizedYelpData.csv"
    5 Create DataFrame with all of the zip code data and save to disk; named "YelpData.csv"
    

In [2]:
#-- Import Libraries
import pandas as pd
import os

In [3]:
#-- Get List of Files on Disk
outputPath = os.path.join(".", "Output")

files = os.listdir(outputPath)


#-- Load into Single DataFrame
yelpData_df = None

for file in files:
    
    if (file.startswith("YelpData_") == True):
        
        #- Get DataFrame
        filePath = os.path.join(".", "Output", file)
    
        zipCodeYelpData_df = pd.read_csv(filePath)
        
        
        if (yelpData_df == None):
            yelpData_df = zipCodeYelpData_df
            
        else:
            yelpData_df = pd.concat(zipCodeYelpData_df, yelpData_df)
        
        
        
       
    

NameError: name 'fileName' is not defined

In [18]:
def getSummarizedYelpData(fileName):
    '''
    
    Accepts : filePath (str) path to the csv that contains the information for the zipcode
    
    Returns : (dictionary) contains the results from Yelp data
                "df" - DataFrame from the csv file
                "summary" - dictionary that contains the summary information
                    "TotalRecords" - total number of records for zip code
                    "Rating_$" 
                    "Rating_$$"
                    "Rating_$$$"
                    "Rating_$$$"
                    "Rating_NA"
                    
    '''
    
    #-- Read File
    filePath = os.path.join(".", "Output", fileName)
    
    yelpData_df = pd.read_csv(filePath)
    
    
    #-- Summarize Data
    
    
    
    
    
    
    #-- Create Result Dictionary
    # Used to return multiple objects to calling function
    results = {
        "df": yelpData_df
    }
    
    return results
